In [2]:
import os
import re
import ast
import pandas as pd

RAW_DIR = r"C:\Users\Andre\Desktop\ML\CdD-ML\dados\raw"
PROC_DIR = r"C:\Users\Andre\Desktop\ML\CdD-ML\dados\processed"

os.makedirs(PROC_DIR, exist_ok=True)


# ------------------------------------------------------------------

def ler(path):
    if not os.path.exists(path):
        print(f"[AVISO] {os.path.basename(path)} não encontrado")
        return None
    return pd.read_csv(path)


def salvar(df, fname):
    out = os.path.join(PROC_DIR, fname)
    df.to_csv(out, index=False)
    print(f"[OK] {fname} salvo")


def limpa_uri(df):
    return df.drop(columns=[c for c in df.columns
                            if "uri" in c.lower() or "url" in c.lower()],
                   errors="ignore")


# ------------------------------------------------------------------
print("\n[DEPUTADOS]")
df = ler(os.path.join(RAW_DIR, "deputados_legislatura_56.csv"))
if df is not None:
    df = (df.rename(columns={"id": "id_deputado",
                             "nome": "nome_deputado"})
            .pipe(limpa_uri))
    salvar(df, "deputados.csv")


# ------------------------------------------------------------------

print("\n[PARTIDOS]")
df = ler(os.path.join(RAW_DIR, "partidos.csv"))
if df is not None:
    df = (df.rename(columns={"id": "id_partido",
                             "sigla": "sigla_partido",
                             "nome": "nome_partido"})
            .pipe(limpa_uri))
    salvar(df, "partidos.csv")


# ------------------------------------------------------------------

def trata_proposicoes(df):
    return (df.rename(columns={"id": "id_proposicao",
                               "siglaTipo": "sigla_tipo",
                               "codTipo": "cod_tipo",
                               "numero": "numero_proposicao"})
              .pipe(limpa_uri))

def trata_votacoes(df):
    df = df.rename(columns={"id": "id_votacao",
                            "siglaOrgao": "sigla_orgao",
                            "dataHoraRegistro": "data_hora_registro",
                            "proposicaoObjeto": "descricao_proposicao"})
    df = limpa_uri(df)
    # garante id_proposicao na 2ª posição
    cols = df.columns.tolist()
    if "id_proposicao" in cols:
        cols.insert(1, cols.pop(cols.index("id_proposicao")))
        df = df[cols]
    return df

def trata_orientacoes(df):
    return limpa_uri(df)

def trata_autores(df):
    return limpa_uri(df).rename(columns={"nome": "nome_autor"})

def trata_temas(df):
    return limpa_uri(df)

TRATADORES = {
    r"proposicoes_(\d{4})\.csv": trata_proposicoes,
    r"votacoes_proposicoes_(\d{4})\.csv": trata_votacoes,
    r"orientacoes_proposicoes_(\d{4})\.csv": trata_orientacoes,
    r"autores_proposicoes_(\d{4})\.csv": trata_autores,
    r"temas_proposicoes_(\d{4})\.csv": trata_temas,
}

for fname in os.listdir(RAW_DIR):
    for padrao, func in TRATADORES.items():
        m = re.fullmatch(padrao, fname)
        if m:
            ano = m.group(1)
            print(f"\n[{fname}]")
            df = ler(os.path.join(RAW_DIR, fname))
            if df is not None:
                df = func(df)
                salvar(df, fname)
            break   # já casou → não precisa verificar outros padrões


# ------------------------------------------------------------------

def trata_votos_individuais(path_1, path_2, ano):
    """Une part 1 + 2 e aplica limpeza."""
    df1, df2 = ler(path_1), ler(path_2)
    if df1 is None or df2 is None:
        return
    df = pd.concat([df1, df2], ignore_index=True)

    df = df.rename(columns={"deputado_": "deputado"}) if "deputado_" in df.columns else df

    # expande JSON da coluna 'deputado'
    def safe(x):
        try:
            return ast.literal_eval(x)
        except Exception:
            return {}

    dep_exp = df["deputado"].apply(safe).apply(pd.Series)
    df = pd.concat([df.drop(columns=["deputado"]), dep_exp], axis=1)

    df = (df.pipe(limpa_uri)
              .drop(columns=["nome", "siglaPartido", "siglaUf",
                             "idLegislatura", "email"],
                    errors="ignore")
              .rename(columns={"id": "id_deputado"}))

    if "tipoVoto" in df.columns:
        df["tipoVoto"] = df["tipoVoto"].map({"Sim": 1, "Não": 0}).fillna(df["tipoVoto"])

    ordem = ["id_votacao", "id_deputado", "tipoVoto", "dataRegistroVoto"]
    df = df[[c for c in ordem if c in df.columns]]

    salvar(df, f"votos_individuais_proposicoes_{ano}.csv")


print("\n[VOTOS INDIVIDUAIS]")
for ano in (2019, 2020, 2021, 2022):
    p1 = os.path.join(RAW_DIR, f"votos_individuais_proposicoes_raw_{ano}.1.csv")
    p2 = os.path.join(RAW_DIR, f"votos_individuais_proposicoes_raw_{ano}.2.csv")
    trata_votos_individuais(p1, p2, ano)

print("\n[PIPELINE FINALIZADO]")



[DEPUTADOS]
[OK] deputados.csv salvo

[PARTIDOS]
[OK] partidos.csv salvo

[autores_proposicoes_2019.csv]
[OK] autores_proposicoes_2019.csv salvo

[autores_proposicoes_2020.csv]
[OK] autores_proposicoes_2020.csv salvo

[autores_proposicoes_2021.csv]
[OK] autores_proposicoes_2021.csv salvo

[autores_proposicoes_2022.csv]
[OK] autores_proposicoes_2022.csv salvo

[orientacoes_proposicoes_2019.csv]
[OK] orientacoes_proposicoes_2019.csv salvo

[orientacoes_proposicoes_2020.csv]
[OK] orientacoes_proposicoes_2020.csv salvo

[orientacoes_proposicoes_2021.csv]
[OK] orientacoes_proposicoes_2021.csv salvo

[orientacoes_proposicoes_2022.csv]
[OK] orientacoes_proposicoes_2022.csv salvo

[proposicoes_2019.csv]
[OK] proposicoes_2019.csv salvo

[proposicoes_2020.csv]
[OK] proposicoes_2020.csv salvo

[proposicoes_2021.csv]
[OK] proposicoes_2021.csv salvo

[proposicoes_2022.csv]
[OK] proposicoes_2022.csv salvo

[proposicoes_2023.csv]
[OK] proposicoes_2023.csv salvo

[temas_proposicoes_2019.csv]
[OK] tem